In [1]:
from kafka import KafkaConsumer

In [2]:
import os
SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.2'
import findspark
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'
findspark.init()
import pyspark
# sc = pyspark.SparkContext(appName="myAppName")
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, IntegerType, StringType, TimestampType, StructField
from pyspark.context import SparkContext
from pyspark.sql import functions as F

In [3]:
spark = SparkSession.builder\
          .master('local[*]')\
          .appName("myAppName")\
          .getOrCreate()

In [4]:
# Reduce logging verbosity
spark.sparkContext.setLogLevel("WARN")
jaas_config = f'kafkashaded.org.apache.kafka.common.security.scram.ScramLoginModule required username="c2V0dGxlZC10ZXJyYXBpbi0xMjUxOCTBb5AEffUiTulATzsbFtDRxbvhkO0Wsnc" password="N2E2ZGVjY2UtZDY4YS00MjM4LTk5NTktMjU1OTRiZWQ4Y2Ix";'

In [5]:
## Kafka configs
kafka_source_config = {
    "kafka.sasl.jaas.config": jaas_config,
    "kafka.bootstrap.servers" : 'settled-terrapin-12518-eu2-kafka.upstash.io:9092',
    "kafka.sasl.mechanism": "SCRAM-SHA-256",
    "kafka.security.protocol" : "SASL_SSL",
    "subscribe": "sec_topic",
    "startingOffsets": "earliest",
    "failOnDataLoss": "false"
}

In [6]:
## Source Schema
df_schema = StructType([
    # StructField("key", StringType(), True),
    StructField("value", StringType(), True)
    # StructField("timestamp", TimestampType(), True),
    # StructField("timestamp", TimestampType(), True)
])

In [7]:
# kafka_sink_config = {
#     "kafka.sasl.jaas.config": jaas_config,
#     "kafka.bootstrap.servers": 'settled-terrapin-12518-eu2-kafka.upstash.io:9092',
#     "kafka.sasl.mechanism": "SCRAM-SHA-256",
#     "kafka.security.protocol": "SASL_SSL",
#     "topic": "sink",
#     "checkpointLocation" : "./checkpoint.txt"
# }

In [8]:
# df = spark.read\
#     .format("kafka")\
#     .option("kafka.bootstrap.servers",'settled-terrapin-12518-eu2-kafka.upstash.io:9092')\
#     .option("subscribe","sec_topic")\
#     .option("startingOffsets","earliest")\
#     .load()

## Read Stream
streaming_df = spark\
    .readStream\
    .format("kafka")\
    .options(**kafka_source_config)\
    .load()

In [9]:
streaming_df

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]

In [10]:
streaming_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [11]:
## Prepare Sink DF
sink_df = streaming_df.selectExpr("CAST(value AS STRING) as value")\
    .select(
        # Convert the value to a string
        F.from_json(F.col("value").cast("string"), df_schema).alias("value")
    )

In [12]:
sink_df.printSchema()

root
 |-- value: struct (nullable = true)
 |    |-- value: string (nullable = true)



In [13]:
query = sink_df.writeStream \
    .format("csv") \
        .option("format", "append")\
        .option("path", "C:/Users/Lenovo/Desktop/spark/New folder/spark_stream_job/stream_output") \
        .option("checkpointLocation", "C:/Users/Lenovo/Desktop/spark/New folder/spark_stream_job/checkpoint") \
        .outputMode("append") \
        .start()

In [15]:
# Start the streaming application to run until the following happens
# 1. Exception in the running program
# 2. Manual Interruption
query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = 0fd5c2c6-49fe-4e45-84ba-74e113ab0d94, runId = 94ab1b07-5077-4a45-babd-972f317437a5] terminated with exception: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'

In [13]:
# Use PySpark to read in all "part" files
# allfiles =  spark.read.option("header","false").csv("C:/Users/Lenovo/Desktop/spark/New folder/spark_stream_job/stream_output/part-*.csv")

In [ ]:
# Output as CSV file
# allfiles.coalesce(1).write.format("csv").option("header", "false").save("C:/Users/Lenovo/Desktop/spark/New folder/spark_stream_job/stream_output/single_csv_file/")

In [62]:
sink_df.show()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
kafka

In [38]:
df1 = df.selectExpr("cast(value as string)")

In [39]:
df1.printSchema()

root
 |-- value: string (nullable = true)



In [40]:
df1.collect()

Py4JJavaError: An error occurred while calling o114.collectToPython.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.util.UninterruptibleThreadRunner.runUninterruptibly(UninterruptibleThreadRunner.scala:46)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.fetchPartitionOffsets(KafkaOffsetReaderAdmin.scala:145)
	at org.apache.spark.sql.kafka010.KafkaOffsetReaderAdmin.getOffsetRangesFromUnresolvedOffsets(KafkaOffsetReaderAdmin.scala:379)
	at org.apache.spark.sql.kafka010.KafkaRelation.buildScan(KafkaRelation.scala:67)
	at org.apache.spark.sql.execution.datasources.DataSourceStrategy$.apply(DataSourceStrategy.scala:349)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:48

In [6]:
# df.select('value').show()

In [3]:
consumer = KafkaConsumer(
    'sec_topic',
    bootstrap_servers='settled-terrapin-12518-eu2-kafka.upstash.io:9092',
    sasl_mechanism='SCRAM-SHA-256',
    security_protocol='SASL_SSL',
    sasl_plain_username='c2V0dGxlZC10ZXJyYXBpbi0xMjUxOCTBb5AEffUiTulATzsbFtDRxbvhkO0Wsnc',
    sasl_plain_password='N2E2ZGVjY2UtZDY4YS00MjM4LTk5NTktMjU1OTRiZWQ4Y2Ix',
    group_id='YOUR_CONSUMER_GROUP',
    auto_offset_reset='earliest'
)

try:
    for message in consumer:
        print(f"Received message: {message.value}")
except KeyboardInterrupt:
    pass
finally:
    consumer.close()

Received message: b'Hello from python'
